In [19]:
# import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
import os
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from keras import Sequential
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [20]:
# load the data
d_set=os.listdir('House_Room_Dataset')
d_set

['Bathroom', 'Bedroom', 'Dinning', 'Kitchen', 'Livingroom']

### we have 5 number of categories in total 

In [21]:
# now we have to extract the

# list to append the images
imgs=[]

# loop to get the columns
for col in d_set:
    # here col means the name of the folder
    all_cols=os.listdir("House_Room_Dataset"+ "/"+ col)

    # loop to get the images from each column and the append them to the above list
    for img in all_cols:
        # append the column name and the image 
        imgs.append((col, str('House_Room_Dataset' +"/"+col) + "/" +img))

# now to print some of the values extracted for understanding
for i in range(10):
    print(imgs[i])
    i+=1

('Bathroom', 'House_Room_Dataset/Bathroom/bath_1001.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1003.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1004.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1005.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1006.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1007.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1010.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1011.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1012.jpg')
('Bathroom', 'House_Room_Dataset/Bathroom/bath_1013.jpg')


### we have extracted the images

In [22]:
# convert the list into a pandas dataframe
img_df=pd.DataFrame(data=imgs, columns=['category', 'image'])

# a look at the head and tail of the dataframe created above
print(img_df.head(5))
print(img_df.tail(5))

   category                                      image
0  Bathroom  House_Room_Dataset/Bathroom/bath_1001.jpg
1  Bathroom  House_Room_Dataset/Bathroom/bath_1003.jpg
2  Bathroom  House_Room_Dataset/Bathroom/bath_1004.jpg
3  Bathroom  House_Room_Dataset/Bathroom/bath_1005.jpg
4  Bathroom  House_Room_Dataset/Bathroom/bath_1006.jpg
        category                                          image
1495  Livingroom  House_Room_Dataset/Livingroom/living_1423.jpg
1496  Livingroom  House_Room_Dataset/Livingroom/living_1424.jpg
1497  Livingroom  House_Room_Dataset/Livingroom/living_1425.jpg
1498  Livingroom  House_Room_Dataset/Livingroom/living_1426.jpg
1499  Livingroom  House_Room_Dataset/Livingroom/living_1427.jpg


In [23]:
# now we will change the size of the extracted images 
# resizing is done to make sure all images are equal and size is according to the architecture
# the lenet5 supports 36, 36 size of images
# for this purpose we will make use of the cv2
import cv2

# define the directory path
dir_path='House_Room_Dataset/'

# define the size for the images
img_size=36

# list for the resized images and another for storing the labels
resized_img=[]
labels=[]

# loop around the dataset to fetch the filenames
for i in d_set:
    data_path=dir_path + str(i)
    filenames=[i for i in os.listdir(data_path)]

    # loop around the filenames to fetch the images and then resize them
    for f in filenames:
        to_resize=cv2.imread(data_path+'/'+f)
        to_resize=cv2.resize(to_resize, (img_size, img_size))
        resized_img.append(to_resize)
        labels.append(i)


In [24]:
# convert the images into an array
images=np.array(resized_img)

# now convert into float
images=images.astype('float32')/255.0
images.shape

(1500, 36, 36, 3)

In [25]:
# since the labes or say the image categories are in the string format like bedroom, bathroom etc, we need to convert them into numbers too
# for this we will make use of the laabelencoder from sklearn aas follows
from sklearn.preprocessing  import LabelEncoder, OneHotEncoder

# since the output will be either bedroom, bathroom, or any of the categories, so our Y means output column will contain the categories
# for this we need to separate the categories as follows and then apply labelencoder
y=img_df['category'].values

# encoder
en=LabelEncoder()
# apply on y
y=en.fit_transform(y)

y

array([0, 0, 0, ..., 4, 4, 4])

In [26]:
# now apply OneHotEncoder to convert the scalar output as mentioned above, in the vector ouptut
# after that the sshape will be like (5250, 5)
y=y.reshape(-1, 1)
ohe=OneHotEncoder()
y=ohe.fit_transform(y).toarray()
y.shape

(1500, 5)

In [27]:
# train test split the data
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# shuffle the images
images, y=shuffle(images, y, random_state=1)

# split the data
X_train,  X_test, y_train, y_test=train_test_split(images, y, test_size=0.2)

In [28]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1200, 36, 36, 3), (1200, 5), (300, 36, 36, 3), (300, 5))

In [34]:
# now make the architecture 
model=Sequential()

# add the layers according to the architecture
# so the first layer is the input convolutional layer with input 28*28*1, 6 filters. the size of conv layer is 5,5 with no padding
model.add(Conv2D(6, kernel_size=(5,5), strides=(1, 1), padding='valid', activation='tanh', input_shape=(36,36,3)))
# now apply the avg pooling on the above layer
model.add(AveragePooling2D(pool_size=(2,2), strides=2, padding='valid'))

# add another conv layer
model.add(Conv2D(16, kernel_size=(5,5), padding='valid', strides=(1, 1), activation='tanh'))
# pooling layer
model.add(AveragePooling2D(pool_size=(2,2), strides=(2, 2), padding='valid'))

# after the pooling we faltten the model
model.add(Flatten())

# fully connected layers
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(Dense(5, activation='softmax'))

# to display the summary of the model
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 6)      │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_6             │ (None, 16, 16, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 12, 12, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_7             │ (None, 6, 6, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 120)            │        69,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           425 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,701 (323.05 KB)

 Trainable params: 82,701 (323.05 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
X_train.shape, y_train.shape

((1200, 36, 36, 3), (1200, 5))

In [36]:
# now compile the model
# from tensorflow.keras.optimizers import  SGD
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [38]:
# now train the model
model.fit(X_train, y_train, epochs=200, batch_size=64)

Epoch 1/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3393 - loss: 1.5165
Epoch 2/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3297 - loss: 1.5220
Epoch 3/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3197 - loss: 1.5332
Epoch 4/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3480 - loss: 1.5031
Epoch 5/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3414 - loss: 1.5122
Epoch 6/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3619 - loss: 1.4969
Epoch 7/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3400 - loss: 1.5089
Epoch 8/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3415 - loss: 1.5052
Epoch 9/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3558 - loss: 1.5064
Epoch 10/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3523 - loss: 1.5145
Epoch 11/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3630 - loss: 1.4917
Epoch 12/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy